## Facebook SQL interview Question

In [ ]:
https://datalemur.com/questions/updated-status

Learnings

You can use the UNION operator to join two columns with the same data types together.

Amazon.com can have a table of customers user_id and their average order value
and another table with customers user_id and a columns stating if they have ever had a prime membership

Amazon wants to know how many paying customers they have (customer swho purchased any product or service)

Not every customer who purchses has a prime membership.

We can use get all the customer_id's from the first table and all the customer_id's from the second table.
Then use the union operator to combine all the customer ids from both tables

In [ ]:
-- My solution
/* Took approximately 45 mins to solve because I didn't know how to use FULL OUTER JOIN 
to get a complete list of companies that advertised on Facebook.

Some companies didn't appear in the advertisers table so INNER JOINS would not give a full
list of all companies that advertised on Facebook
*/

-- Get all users all in one column
WITH all_users AS (
  SELECT user_id
  FROM advertiser

  UNION

  SELECT user_id
  FROM daily_pay
),

-- Join all_users, advertiser and daily paid table
ad_dpay2 AS (
  SELECT 
        au.user_id AS main_id,
        a.user_id,
        dp.user_id,
        a.status,
        dp.paid
        
  FROM all_users AS au 
  
  LEFT JOIN advertiser AS a 
  ON au.user_id = a.user_id

  LEFT JOIN daily_pay AS dp 
  ON au.user_id = dp.user_id

),

-- Create new column to get new status of advertisers
user_new_status AS (
  SELECT 
      main_id,
      status,
      paid,
      CASE
        WHEN status IN ('NEW', 'EXISTING', 'RESURRECT', 'CHURN') AND paid IS NULL THEN 'CHURN'
        WHEN status IN ('NEW', 'EXISTING', 'RESURRECT') AND paid >= 0 THEN 'EXISTING'
        WHEN status IN ('CHURN') AND paid >0 THEN 'RESURRECT'
        ELSE 'NEW'
      END AS new_status
  FROM ad_dpay2
)


-- Display new_status of users
SELECT main_id, new_status
FROM user_new_status
ORDER BY main_id;

In [ ]:
  -- Example iof UNION operator
  SELECT user_id
  FROM advertiser

  UNION

  SELECT user_id
  FROM daily_pay

### More Efficient solution


In [ ]:
-- Step 1: Merge tables using FULL OUTER JOIN
SELECT *
FROM advertiser
FULL OUTER JOIN daily_pay
ON advertiser.user_id = daily_pay.user_id;

-- Step 2: Assign payment status based on payment
SELECT   
  CASE 
    WHEN paid IS NULL THEN 'CHURN' 
    WHEN paid IS NOT NULL AND advertiser.status IN ('NEW','EXISTING','RESURRECT') THEN 'EXISTING'
    WHEN paid IS NOT NULL AND advertiser.status = 'CHURN' THEN 'RESURRECT'
    WHEN paid IS NOT NULL AND advertiser.status IS NULL THEN 'NEW'
  END AS new_status
FROM advertiser
FULL OUTER JOIN daily_pay
  ON advertiser.user_id = daily_pay.user_id;

-- Step 3: Select the user IDs with adjusted status
  SELECT 
  COALESCE(advertiser.user_id, daily_pay.user_id) AS user_id,
  CASE 
    WHEN paid IS NULL THEN 'CHURN' 
    WHEN paid IS NOT NULL AND advertiser.status IN ('NEW','EXISTING','RESURRECT') THEN 'EXISTING'
    WHEN paid IS NOT NULL AND advertiser.status = 'CHURN' THEN 'RESURRECT'
    WHEN paid IS NOT NULL AND advertiser.status IS NULL THEN 'NEW'
  END AS new_status
FROM advertiser
FULL OUTER JOIN daily_pay
  ON advertiser.user_id = daily_pay.user_id
ORDER BY user_id;

#### Errors I ran into and solved

If you are joining multiple tables and each table has columns that have the same name and wrapping the join in a CTE, it is advisable to give the namesake column you wish to primarily use an alias so you don't run in error later when you call the namesake column in another query.

